In [30]:
# TODO: pip install pymysql requests beautifulsoup4 tqdm
# 豆瓣源
# TODO: pip install pymysql requests beautifulsoup4 tqdm -i https://pypi.douban.com/simple
# 清华大学源
# TODO: pip install pymysql requests beautifulsoup4 tqdm -i https://pypi.tuna.tsinghua.edu.cn/simple

In [31]:
import pymysql

# def replaceNT(content: str) -> str:
#   return content.replace("\n", "").replace('\t', "")

replaceNT = lambda content : content.replace("\n", "").replace('\t', "")

def connectMySQL(insertData):
    def wrapper(title: str, item: dict):
        # 连接数据库
        try:
            connect = pymysql.connect(
                host='localhost',
                user='root',
                password='root',
                port=3306,
                db='photoworldForPython',  # 指定数据库
                autocommit=True,  # 自动提交到数据库
                auth_plugin_map="mysql_native_password"
            )
            # print("Databases Connect Success")
        
            db = connect.cursor(pymysql.cursors.DictCursor)  # 创建游标，查询数据以字典形式返回，默认以元组形式返回
            
            insertData(db, title, item)
            
            db.close()  # 关闭数据库
        except Exception as e:
            print("Databases Connect Error:", e)
            pass
    return wrapper
@connectMySQL
def insertData(db, title: str, item: dict) -> None:
    title=title,
    imgURL = item['imgurl'],
    content = item['excerpt'],
    tags = ', '.join(item['tags']),
    time = item['Meta']['recent_time']['datetime'],
    type = item['Meta']['recent_cat']['content'],
    author = item['Meta']['recent_author']['content'],
    detailsURL = item['link']
    mysqlStatement = f'''
                INSERT INTO PageForImage (title, imgURL, content, tags, time, type, author, detailsURL)
                VALUES ('{title}', '{imgURL}', '{content}', '{tags}', '{time}', '{type}', '{author}', '{detailsURL}');
            '''
    db.execute(mysqlStatement)

In [32]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time


WAIT_TIME_SECONDS = 0  # 设置默认的等待时间
# url = "https://www.photoworld.com.cn/"
url = "https://www.photoworld.com.cn/category/images"
# 获取页数
def get_page_num(url: str):
    try:
        response = requests.get(url)
        response.encoding = 'UTF-8'
        
        # 检查请求是否成功
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            page_num = soup.find_all('a', {'class' : 'page-numbers'})[-2].string
            return int(page_num)
        else:
            print(f"请求失败，状态码：{response.status_code}")
            return None
    except Exception as e:
        print(f"请求出错，错误信息：{e}")
        return None
page_num = get_page_num(url)
print(page_num)

11


In [33]:
page_resultArray = []
for num in tqdm(range(1,page_num+1)):
    page_url = url + f'/page/{num}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser') if response.status_code == 200 else None
    articles = soup.find_all('article')
    resultArray = []
    for article in articles:
        data = {}
        try:
          imgurl = article.find('a').find('img').get('src')
          link = article.find('h3').find('a').get('href')
          title = replaceNT(article.find('h3').find('a').text)
          excerpt = replaceNT(article.find('p', {'class': 'big-thumb-excerpt'}).text)
          tags = [i.text for i in article.find('p', {'class': 'tag'}).find_all('a')]
          recent_time = {'time': replaceNT(article.find('time', {'class': 'recent-time'}).text), 'datetime': article.find('time', {'class': 'recent-time'}).get('datetime')}
          recent_cat = {
              'content': article.find('span', {'class': 'recent-cat'}).text.replace("\n", ""),
                        'url': article.find('span', {'class': 'recent-cat'}).find('a').get('href')
                        }
          recent_author = {
              'content': article.find('span', {'class': 'recent-author'}).text.replace("\n", ""),
              'url': article.find('span', {'class': 'recent-author'}).find('a').get('href')
          }
          data[title] = {
                  "imgurl": imgurl,
                  "excerpt": excerpt,
                  "tags": tags,
                  "Meta": {
                      "recent_time": recent_time,
                      "recent_cat": recent_cat,
                      "recent_author": recent_author
                  },
                  "link": link
              }
          resultArray.append(data)
        except Exception as e:
          print("append data Error", e)
          resultArray.append("None")
        page_resultArray.append(resultArray)
        time.sleep(WAIT_TIME_SECONDS)
#     print(len(resultArray),end="===" + page_url)
print(len(page_resultArray))

100%|██████████| 11/11 [00:07<00:00,  1.51it/s]


In [36]:
a = 0
b = 0
c = 0
for page_items in tqdm(page_resultArray):
    a = a + 1
    for item in page_items:
        b = b + 1
        for title, data in item.items():
            c = c + 1
            insertData(title=title, item=data)
print(f"{a},{b},{c}")

100%|██████████| 220/220 [00:03<00:00, 65.02it/s]

220,4400,4400
